### Exercise: Dot products and cosines

One very useful property of the dot product allows us to find the angle between two vectors. For two vectors a and b,

dot(a, b) = ||a|| ||b|| cos(theta)

Here, ||a|| is the length of the vector a.

This property is so fundamental that it's considered an [alternative definition of the dot product](https://en.wikipedia.org/wiki/Dot_product#Geometric_definition). It is widely used in lighting calculations for computer graphics.

Use this property to find the angle between the two vectors (1,2,3) and (-3, 2, -1). You'll probably want to make use of the inverse cosine function (also known as arcos), which in numpy is `np.arccos()`.

In [6]:
import numpy as np

vector_a = np.array([1,2,3])
vector_b = np.array([-3,2,-1])

# Work out angle between these vectors, and print it.
len_a = np.linalg.norm(vector_a)
len_b = np.linalg.norm(vector_b)
angle = np.arccos(np.dot(vector_a, vector_b) / (len_a * len_b))
print(angle)

1.714143895700262


### Exercise: Cross Product and Parallelograms

Another property of the cross product of vectors a and b is that its magnitude (i.e. length) is equal to that of a parallelogram with the vectors a and b as two of the sides, as shown in this diagram (from [wikipedia](https://en.wikipedia.org/wiki/Cross_product#/media/File:Cross_product_parallelogram.svg))

![Diagram](https://upload.wikimedia.org/wikipedia/commons/4/4e/Cross_product_parallelogram.svg)

I'd like you to verify this for the vectors a and b below. Do this by dividing the parallelogram into two triangles, and working out the area of each. The length of the sides of these triangles will be ||a||, ||b|| and ||b - a||. To work out their area you may want to make use of [Heron's formula](https://en.wikipedia.org/wiki/Heron%27s_formula).

In [4]:
import numpy as np

vector_a = np.array([1,2,3])
vector_b = np.array([-3,2,-1])

cross_product = np.cross(vector_a, vector_b)
cross_area = np.linalg.norm(cross_product) # This function works out the norm, i.e. the length of the vector in this case.

print("Area estimated with cross product", cross_area)

# Work out the lengths of the 3 sides of the triangle
# Find the area of each triangle using heron's formula
# Double to get the parallelogram area.

# Find length of each side
len_a = np.linalg.norm(vector_a)
len_b = np.linalg.norm(vector_b)
len_c = np.linalg.norm(vector_b - vector_a)
# Find semi-perimeter
semi_perimeter = (len_a + len_b + len_c) / 2
# Use heron's formula to find area of one of the triangles, and multiply by 2 to get the final answer.
triangle_area = 2 * np.sqrt(semi_perimeter * (semi_perimeter - len_a) * (semi_perimeter - len_b) * (semi_perimeter - len_c)) 

## This should be the same as cross_area above.
print("Area estimated by finding triangle areas", triangle_area)

Area estimated with cross product 13.856406460551018
Area estimated by finding triangle areas 13.856406460551018


### Exercise - Pseudoinverse

Non-square matrices don't have true inverses, but the SVD can be used to find a pseudoinverse (also called the [Moore-Penrose Inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse)) of any matrix. This has important applications in solving least-squares problems, amongst other things.

For a matrix `A`, the pseudoinverse `pinv(A)` doesn't behave exactly like a true inverse, but does satisfy some conditions:

1. `A @ pinv(A) @ A == A`
2. `pinv(A) @ A @ pinv(A) == pinv(A)`
3. `pinv(A) @ A == I` *only* if the columns of A are linearly independent
4. `A @ pinv(A) == I` *only* if the rows of A are linearly independent

Look up the formula to compute the pseudoinverse of the matrix here, and check these conditions hold.

In [25]:
import numpy as np

matrix = np.array([
    [0, 1],
    [1, 2],
    [2, 3]
])

u, s, vt = np.linalg.svd(matrix)

# Construct the matrix sigma
sigma = np.zeros((u.shape[1], s.shape[0]))
for i in range(s.shape[0]):
    sigma[i, i] = s[i]

# Find the pseudoinverse of matrix
# Find pseudoinverse of sigma
sigmapinv = sigma.transpose()
# Replace nonzero elements with multiplicative inverse
sigmapinv[sigmapinv != 0] = 1 / sigmapinv[sigmapinv != 0]
# Now compute the pseudoinverse of matrix
matrixpinv = vt.transpose() @ sigmapinv @ u.transpose()
print("The matrix pseudo inverse is\n", matrixpinv)



The matrix pseudo inverse is
 [[-1.33333333 -0.33333333  0.66666667]
 [ 0.83333333  0.33333333 -0.16666667]]


In [24]:

# Test the 4 conditions
print("Checking condition 1")
print("matrix @ pinv(matrix) @ matrix\n", matrix @ matrixpinv @ matrix)
print("(matrix @ pinv(matrix) @ matrix) == matrix:", np.allclose(matrix @ matrixpinv @ matrix, matrix))
print()
print("Checking condition 2")
print("pinv(matrix) @ matrix @ pinv(matrix)\n", matrixpinv @ matrix @ matrixpinv)
print("(pinv(matrix) @ matrix @ pinv(matrix)) == matrix:", np.allclose(matrixpinv @ matrix @ matrixpinv, matrixpinv))
print()
print("Checking condition 3")
print("pinv(matrix) @ matrix\n", matrixpinv @ matrix)
print("pinv(matrix) @ matrix == I:", np.allclose(matrixpinv @ matrix, np.eye(2)))
# Condition 3 does hold, as the columns are linearly independent
print()
print("Checking condition 4")
print("matrix @ pinv(matrix)\n", matrix @ matrixpinv)
print("(matrix @ pinv(matrix)) == I:", np.allclose(matrix @ matrixpinv, np.eye(3)))
# Note condition 4 does not hold
# This is because the rows aren't linearly independent
# It's not possible to have more than 2 vectors that are linearly independent in 2D
# Or more specifically, you can express row 2 as a linear combination of rows 0 and 1, like this:
print()
print("Linear combination:", -matrix[0,:] + 2*matrix[1,:])
print("Checking if row 2 is a linear combination of rows 0 and 1:", np.allclose(-matrix[0,:] + 2*matrix[1,:], matrix[2,:]))
# Generally, if your matrix isn't square at most one of conditions 3 & 4 can hold

Checking condition 1
matrix @ pinv(matrix) @ matrix
 [[-1.66533454e-16  1.00000000e+00]
 [ 1.00000000e+00  2.00000000e+00]
 [ 2.00000000e+00  3.00000000e+00]]
(matrix @ pinv(matrix) @ matrix) == matrix: True

Checking condition 2
pinv(matrix) @ matrix @ pinv(matrix)
 [[-1.33333333 -0.33333333  0.66666667]
 [ 0.83333333  0.33333333 -0.16666667]]
(pinv(matrix) @ matrix @ pinv(matrix)) == matrix: True

Checking condition 3
pinv(matrix) @ matrix
 [[ 1.00000000e+00  0.00000000e+00]
 [-1.66533454e-16  1.00000000e+00]]
pinv(matrix) @ matrix == I: True

Checking condition 4
matrix @ pinv(matrix)
 [[ 0.83333333  0.33333333 -0.16666667]
 [ 0.33333333  0.33333333  0.33333333]
 [-0.16666667  0.33333333  0.83333333]]
(matrix @ pinv(matrix)) == I: False

Linear combination: [2 3]
Checking if row 2 is a linear combination of rows 0 and 1: True
